# Table Of Contents 
1. [Describe the goals of the study](#1)
2. [Study the tables (print the first rows)](#2)
3. [Tasks](#3)
  * [Find the number of books released after January 1, 2000.](#4)
  * [Find the number of user reviews and the average rating for each book.](#5)
  * [Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).](#6)
  * [Identify the author with the highest average book rating: look only at books with at least 50 ratings.](#7)
  * [Find the average number of text reviews among users who rated more than 50 books.](#8)

<a id="1"></a> 
# Describe the goals of the study

In this project I will dive into the database we have on books, publishers, authors, and customer ratings and reviews of books, in order to generate a value proposition for a new books.

<a id="2"></a> 
# Study the tables (print the first rows)

In [74]:
#!pip install ipython-sql

In [75]:
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine

In [76]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [77]:
pd.io.sql.read_sql('select * from books limit 5', con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [78]:
pd.io.sql.read_sql('select * from authors limit 5', con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [79]:
pd.io.sql.read_sql('select * from ratings limit 5', con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [80]:
pd.io.sql.read_sql('select * from reviews limit 5', con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [81]:
pd.io.sql.read_sql('select * from publishers limit 5', con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<a id="3"></a> 
# Tasks

<a id="4"></a> 
## Find the number of books released after January 1, 2000.

In [82]:
pd.io.sql.read_sql("select COUNT(DISTINCT book_id) AS cnt from books where publication_date > '2000-01-01'", con = engine)

,cnt
0,819


819 books released after January 1, 2000.

<a id="5"></a> 
## Find the number of user reviews and the average rating for each book.

In [83]:
pd.io.sql.read_sql('select books.title, AVG(ratings.rating) AS average_rating, COUNT(DISTINCT reviews.review_id) AS cnt_reviews from ratings LEFT OUTER JOIN reviews ON ratings.username=reviews.username and ratings.book_id=reviews.book_id LEFT OUTER JOIN books ON ratings.book_id=books.book_id group by books.title', con = engine)

,title,average_rating,cnt_reviews
0,'Salem's Lot,3.666667,2
1,1 000 Places to See Before You Die,2.500000,1
2,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,1491: New Revelations of the Americas Before C...,4.500000,2
4,1776,4.000000,4
...,...,...,...
994,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3
995,Xenocide (Ender's Saga #3),3.400000,3
996,Year of Wonders,3.200000,4
997,You Suck (A Love Story #2),4.500000,2


<a id="6"></a> 
## Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [84]:
pd.io.sql.read_sql("select publishers.publisher, COUNT(DISTINCT books.book_id) AS number_of_books from  books LEFT OUTER JOIN publishers ON books.publisher_id=publishers.publisher_id WHERE books.num_pages>50 GROUP BY publishers.publisher ORDER BY number_of_books DESC LIMIT 1", con = engine)

,publisher,number_of_books
0,Penguin Books,42


"Penguin Books" has released the greatest number of books with more than 50 pages (42 books). 

<a id="7"></a> 
## Identify the author with the highest average book rating: look only at books with at least 50 ratings.

In [85]:
pd.io.sql.read_sql("select Sub.author, AVG(Sub.average_rating) AS average_rating_author from (select authors.author, ratings.book_id, AVG(ratings.rating) AS average_rating, COUNT(DISTINCT ratings.rating_id) AS cnt_rating from ratings LEFT OUTER JOIN books ON ratings.book_id=books.book_id LEFT OUTER JOIN authors ON books.author_id=authors.author_id GROUP BY authors.author, ratings.book_id HAVING COUNT(ratings.rating_id)>50 ORDER BY AVG(ratings.rating) DESC) AS Sub group by author ORDER BY AVG(Sub.average_rating) DESC limit 1", con = engine)

,author,average_rating_author
0,J.K. Rowling/Mary GrandPré,4.283844


The author with the highest average book rating (books with at least 50 ratings) is J.K. Rowling! make perfect sense :)

<a id="8"></a> 
## Find the average number of text reviews among users who rated more than 50 books.

In [86]:
pd.io.sql.read_sql('select AVG(Sub.cnt_reviews) AS average from (select ratings.username,COUNT(DISTINCT ratings.book_id) AS cnt_rating, COUNT(DISTINCT reviews.review_id) AS cnt_reviews from ratings LEFT OUTER JOIN reviews ON ratings.username=reviews.username and ratings.book_id=reviews.book_id group by ratings.username having COUNT(DISTINCT ratings.book_id)>50) AS Sub', con = engine)

,average
0,24.333333


The average number of text reviews among users who rated more than 50 books is 24 reviews.